<a href="https://colab.research.google.com/github/KShinyShamma/Activity_Recognition/blob/main/Training_keypoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ultralytics
!pip install opencv-python
import ultralytics
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import os

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 26.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Get Training Data

In [2]:
model = YOLO("yolo11n-pose.pt")

# Activity
activity_videos = {
    'Sitting': ['/content/drive/MyDrive/P2/DATA/Sitting/001.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/002.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/003.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/004.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/005.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/006.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/007.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/008.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/009.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/010.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/011.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/012.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/013.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/014.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/015.mp4'
                ],
    'Laying': ['/content/drive/MyDrive/P2/DATA/Laying/001.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/002.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/003.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/004.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/005.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/006.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/007.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/008.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/009.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/010.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/011.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/012.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/013.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/014.mp4',
                '/content/drive/MyDrive/P2/DATA/Laying/015.mp4'
                ],
    'Jumping': ['/content/drive/MyDrive/P2/DATA/Jumping/001.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/002.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/003.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/004.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/005.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/006.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/007.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/008.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/009.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/010.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/011.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/012.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/013.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/014.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/015.mp4'
                ],
    'Standing': ['/content/drive/MyDrive/P2/DATA/Standing/001.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/002.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/003.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/004.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/005.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/006.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/007.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/008.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/009.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/010.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/011.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/012.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/013.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/014.mp4',
                '/content/drive/MyDrive/P2/DATA/Standing/015.mp4'
                ],
}

# List to store keypoints data for all activities
all_activity_points = []

# LOop function for each activity
for activity, video_paths in activity_videos.items():
    for idx, video_path in enumerate(video_paths):
        cap = cv2.VideoCapture(video_path)

        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        print(fps)
        frames = 5

        # Set up video writer to save output video
        output_video_path = f'/content/drive/MyDrive/P2/YOLO/output01/Train/{activity}_pose_estimation_output_{idx + 1}.mp4'
        #output_video_path = f'/content/drive/MyDrive/P2/YOLO/output01/{activity}_pose_estimation_output.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mpv4')  # Codec
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

        i = 0
        a = 0

        while cap.isOpened():
            flag, frame = cap.read()

            if not flag:
                break

            # Perform YOLO pose estimation on the frame
            results = model(frame, verbose=False)

            for r in results:
                bound_box = r.boxes.xyxy
                conf = r.boxes.conf.tolist()
                keypoints = r.keypoints.xyn.tolist()

                for index, box in enumerate(bound_box):
                    if conf[index] > 0.75:
                        # Draw bounding box around detected activity (the person)
                        x1, y1, x2, y2 = box.tolist()
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

                        # Optionally add a label for the activity
                        label = f'{activity}'
                        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                        # For skeleton??
                        for kp_index, (x, y) in enumerate(keypoints[index]):
                            cv2.circle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])), 5, (0, 255, 0), -1)
                        skeleton_connections = [
                            (0, 1), (1, 2), (2, 3), (3, 4),  # Left arm
                            (0, 5), (5, 6), (6, 7), (7, 8),  # Right arm
                            (9, 10), (10, 11), (11, 12),      # Left leg
                            (9, 13), (13, 14), (14, 15),      # Right leg
                            (0, 9)                            # Spine
                        ]
                        for p1, p2 in skeleton_connections:
                            x1, y1 = keypoints[index][p1]
                            x2, y2 = keypoints[index][p2]
                            cv2.line(frame, (int(x1 * frame.shape[1]), int(y1 * frame.shape[0])),
                                     (int(x2 * frame.shape[1]), int(y2 * frame.shape[0])), (0, 0, 255), 2)

                        # Prepare keypoints data along with the activity label
                        data = {'image_name': f'{activity}_person_{a}.jpg', 'activity': activity}
                        for j in range(len(keypoints[index])):
                            data[f'x{j}'] = keypoints[index][j][0]
                            data[f'y{j}'] = keypoints[index][j][1]


                        all_activity_points.append(data)
                        a += 1

            out.write(frame)

            i += 1

        cap.release()
        out.release()
        print(f'Video saved: {output_video_path}')

# keypoints excel
df = pd.DataFrame(all_activity_points)
csv_file_path = '/content/drive/MyDrive/P2/DATA/activity_Trainkeypoints_02.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data successfully saved to {csv_file_path}")


100%|██████████| 5.97M/5.97M [00:00<00:00, 123MB/s]


30.0
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_1.mp4
30.002752546105146
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_2.mp4
29.257854171532454
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_3.mp4
30.0
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_4.mp4
30.0
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_5.mp4
30.0
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_6.mp4
30.0
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_7.mp4
30.0
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_8.mp4
30.109113368103213
Video saved: /content/drive/MyDrive/P2/YOLO/output01/Train/Sitting_pose_estimation_output_9.mp4
29.97002997002997
Video saved: /content/drive/MyD